In [2]:
!git clone https://github.com/dariush-bahrami/character-tokenizer.git

Cloning into 'character-tokenizer'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 16 (delta 3), reused 12 (delta 3), pack-reused 0
Receiving objects: 100% (16/16), 5.13 KiB | 1.28 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [6]:
# !pip install transformers

In [3]:
import numpy as np
device = 'cuda'

In [4]:
import string
import sys
sys.path.append("/kaggle/working/character-tokenizer")
from charactertokenizer import CharacterTokenizer

chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

In [5]:
example = "Привет"
tokens = tokenizer(example)
print(tokens)

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
tokenizer.convert_ids_to_tokens(tokens['input_ids'])

['[CLS]', 'П', 'р', 'и', 'в', 'е', 'т', '[SEP]']

Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

1. Напишите класс для Dataset/Dataloder и азбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
2. Попробуйте несколько моделей: Bert, Albert, Deberta. (3 балла)
Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [7]:
# !unzip all_accents.zip

In [8]:
import pandas as pd

df = pd.read_csv('/kaggle/input/ska-sdelai-dataset-mraz/all_accents/all_accents.tsv', delimiter='\t')
df.shape[0]

1680534

In [9]:
df.sample(10)

,-де,-д^е
818907,обмачивается,обм^ачивается
753609,невесомое,невес^омое
826637,обосновываемая,обосн^овываемая
917176,отретушированное,отретуш^ированное
1162063,приладожскую,прил^адожскую
1020762,побешусь,побеш^усь
1304621,расценкам,расц^енкам
89748,бессмыслен,бессм^ыслен
1457744,субарендаторов,субаренд^аторов
904182,отксерим,откс^ерим


In [10]:
df['target_idx'] = [word.find('^')for word in df['-д^е']]
df.sample(5)

,-де,-д^е,target_idx
901705,отклеен,откл^еен,4
1212848,пропивай,пропив^ай,6
1595625,харьковчане,харьковч^ане,8
1076979,поконченная,пок^онченная,3
1474462,тактильно,такт^ильно,4


In [11]:
df.drop(index=np.where(df.target_idx == -1)[0], inplace=True)
df.shape[0]

1680027

In [12]:
train_words = df['-де']
target = df.target_idx

In [13]:
max_len = 0
for word in train_words:
    input_ids = tokenizer(word)['input_ids']
    max_len = max(max_len, len(input_ids))
print('Max word length: ', max_len)

Max word length:  58


In [14]:
import torch
from tqdm.notebook import tqdm

In [15]:

# Tokenize all of the words and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
MAX_LENGTH = max_len
# For every sentence...
for word in tqdm(train_words):
    encoded_dict = tokenizer(
                        word,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LENGTH,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


  0%|          | 0/1680027 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2352: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [16]:
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(target.values)


In [17]:
i = 10000
print('Original: ', train_words[i])
print('Token IDs:', input_ids[i])

Original:  агарак
Token IDs: tensor([ 0,  8, 14,  8, 42,  8, 30,  8, 12,  8, 36,  1,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4])


In [18]:
from torch.utils.data import TensorDataset, random_split
import pickle

if input() == '0':
    dataset = TensorDataset(input_ids, attention_masks, labels)
    with open('dataset.pkl', 'wb') as f:
        pickle.dump(dataset, f)
else:
    max_len = 58
    with open('dataset.pkl', 'rb') as f:
        dataset = pickle.load(f)

 0


In [19]:

train_size = int(0.5 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

840,013 training samples
840,014 validation samples


In [20]:
import torch
torch.cuda.empty_cache()

In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

### Train Function

In [22]:
import time
import datetime
import numpy as np

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [23]:
import random

def train_model(model, optimizer, scheduler, epochs, val_step=True):
    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []

    total_t0 = time.time()

    for epoch_i in range(0, epochs):
        # train
        print('\n======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        t0 = time.time()
        total_train_loss = 0

        model.train()

        for step, batch in enumerate(train_dataloader):
            if step % 100 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()        

            output = model(b_input_ids, token_type_ids=None,
                          attention_mask=b_input_mask, labels=b_labels)

            total_train_loss += output.loss.item()
            output.loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)            
        training_time = format_time(time.time() - t0)
        print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
            
        # val
        if not val_step:
            continue
        print("\nRunning Validation...")

        t0 = time.time()
        model.eval()

        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            with torch.no_grad():        
                output = model(b_input_ids, token_type_ids=None, 
                              attention_mask=b_input_mask, labels=b_labels)
                
            total_eval_loss += output.loss.item()

            logits = output.logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_eval_accuracy += flat_accuracy(logits, label_ids)
            

        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        avg_val_loss = total_eval_loss / len(validation_dataloader)
        validation_time = format_time(time.time() - t0)
        
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [24]:
def test_model(model):
    model.eval()
    samples = df.sample(10)
    pred = []
    with torch.no_grad():
        for i in range(10):
            word = samples.iloc[i]['-де']
            input = torch.tensor(tokenizer(word)['input_ids']).to(device)
            output = model(input.unsqueeze(0))
            preds = output.logits.detach().cpu().numpy()
            ans = np.argmax(preds, axis=1)[0]
            pred.append(word[:ans] + '^' + word[ans:])
    samples['prediction'] = pred
    samples['is_right'] = [int(inp == out) for inp, out in zip(samples['-д^е'], pred)]
    return samples

### BERT

In [25]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup


model = BertForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny", 
    num_labels = max_len - 1,   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']

- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

Some weights of BertForSequenceClassification were no

In [26]:
train_model(model, optimizer, scheduler, epochs)



======== Epoch 1 / 4 ========

Training...

  Batch    40  of  3,282.    Elapsed: 0:00:04.

  Batch    80  of  3,282.    Elapsed: 0:00:08.

  Batch   120  of  3,282.    Elapsed: 0:00:11.

  Batch   160  of  3,282.    Elapsed: 0:00:15.

  Batch   200  of  3,282.    Elapsed: 0:00:19.

  Batch   240  of  3,282.    Elapsed: 0:00:23.

  Batch   280  of  3,282.    Elapsed: 0:00:27.

  Batch   320  of  3,282.    Elapsed: 0:00:31.

  Batch   360  of  3,282.    Elapsed: 0:00:35.

  Batch   400  of  3,282.    Elapsed: 0:00:39.

  Batch   440  of  3,282.    Elapsed: 0:00:43.

  Batch   480  of  3,282.    Elapsed: 0:00:46.

  Batch   520  of  3,282.    Elapsed: 0:00:50.

  Batch   560  of  3,282.    Elapsed: 0:00:54.

  Batch   600  of  3,282.    Elapsed: 0:00:58.

  Batch   640  of  3,282.    Elapsed: 0:01:02.

  Batch   680  of  3,282.    Elapsed: 0:01:06.

  Batch   720  of  3,282.    Elapsed: 0:01:10.

  Batch   760  of  3,282.    Elapsed: 0:01:14.

  Batch   800  of  3,282.    Elapsed: 0:01

accuracy: 0.79

In [50]:
torch.save(model.state_dict(), 'rubert_weights.pth')

In [51]:
model.load_state_dict(torch.load('rubert_weights.pth'))

<All keys matched successfully>

In [49]:
test_model(model)

,-де,-д^е,target_idx,prediction,is_right
177319,возродившеюся,возрод^ившеюся,6,возрод^ившеюся,1
293072,гурвиц,г^урвиц,1,г^урвиц,1
1294652,рассердивший,рассерд^ивший,7,рассерд^ивший,1
141466,верталось,верт^алось,4,верт^алось,1
1600328,хлипкому,хл^ипкому,2,хл^ипкому,1
1002135,перфораторного,перфор^аторного,6,перфор^аторного,1
311153,дергавшиеся,д^ергавшиеся,1,дерг^авшиеся,0
606041,лавинообразную,лавинообр^азную,9,лавинообр^азную,1
663080,месящей,мес^ящей,3,мес^ящей,1
1623480,черствеешь,черств^еешь,6,черств^еешь,1


### Albert

In [25]:
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup


model = AutoModelForSequenceClassification.from_pretrained(
    "albert-base-v2", 
    num_labels = max_len - 1,   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [26]:
torch.cuda.empty_cache()

In [27]:
train_model(model, optimizer, scheduler, epochs, val_step=False)


======== Epoch 1 / 1 ========
Training...
  Batch   100  of  13,126.    Elapsed: 0:00:59.
  Batch   200  of  13,126.    Elapsed: 0:01:58.
  Batch   300  of  13,126.    Elapsed: 0:03:00.
  Batch   400  of  13,126.    Elapsed: 0:04:00.
  Batch   500  of  13,126.    Elapsed: 0:05:02.
  Batch   600  of  13,126.    Elapsed: 0:06:03.
  Batch   700  of  13,126.    Elapsed: 0:07:04.
  Batch   800  of  13,126.    Elapsed: 0:08:05.
  Batch   900  of  13,126.    Elapsed: 0:09:06.
  Batch 1,000  of  13,126.    Elapsed: 0:10:07.
  Batch 1,100  of  13,126.    Elapsed: 0:11:08.
  Batch 1,200  of  13,126.    Elapsed: 0:12:09.
  Batch 1,300  of  13,126.    Elapsed: 0:13:10.
  Batch 1,400  of  13,126.    Elapsed: 0:14:12.
  Batch 1,500  of  13,126.    Elapsed: 0:15:13.
  Batch 1,600  of  13,126.    Elapsed: 0:16:14.
  Batch 1,700  of  13,126.    Elapsed: 0:17:15.
  Batch 1,800  of  13,126.    Elapsed: 0:18:16.
  Batch 1,900  of  13,126.    Elapsed: 0:19:18.
  Batch 2,000  of  13,126.    Elapsed: 0:20:1

In [28]:
torch.save(model.state_dict(), 'albert_weights.pth')

In [29]:
model.load_state_dict(torch.load('albert_weights.pth'))

<All keys matched successfully>

In [30]:
test_model(model)

,-де,-д^е,target_idx,prediction,is_right
331232,додумывавшими,дод^умывавшими,3,дод^умывавшими,1
647800,маринова,мар^инова,3,мар^инова,1
431832,зарядившимся,заряд^ившимся,5,заряд^ившимся,1
424960,заполучаем,заполуч^аем,7,заполуч^аем,1
1643854,шлихи,шлих^и,4,шл^ихи,0
442768,затечешь,затеч^ешь,5,затеч^ешь,1
563471,комплементе,комплем^енте,7,комплем^енте,1
819140,обмельчавшей,обмельч^авшей,7,обмельч^авшей,1
1653022,щурившим,щ^урившим,1,щ^урившим,1
1363772,сдружаетесь,сдруж^аетесь,5,сдруж^аетесь,1


### Deberta

In [40]:
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup


model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-small", 
    num_labels = max_len - 1,   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [41]:
train_model(model, optimizer, scheduler, epochs, val_step=False)


======== Epoch 1 / 1 ========
Training...
  Batch   100  of  13,126.    Elapsed: 0:00:41.
  Batch   200  of  13,126.    Elapsed: 0:01:22.
  Batch   300  of  13,126.    Elapsed: 0:02:04.
  Batch   400  of  13,126.    Elapsed: 0:02:45.
  Batch   500  of  13,126.    Elapsed: 0:03:27.
  Batch   600  of  13,126.    Elapsed: 0:04:08.
  Batch   700  of  13,126.    Elapsed: 0:04:49.
  Batch   800  of  13,126.    Elapsed: 0:05:31.
  Batch   900  of  13,126.    Elapsed: 0:06:12.
  Batch 1,000  of  13,126.    Elapsed: 0:06:54.
  Batch 1,100  of  13,126.    Elapsed: 0:07:35.
  Batch 1,200  of  13,126.    Elapsed: 0:08:17.
  Batch 1,300  of  13,126.    Elapsed: 0:08:58.
  Batch 1,400  of  13,126.    Elapsed: 0:09:40.
  Batch 1,500  of  13,126.    Elapsed: 0:10:21.
  Batch 1,600  of  13,126.    Elapsed: 0:11:02.
  Batch 1,700  of  13,126.    Elapsed: 0:11:44.
  Batch 1,800  of  13,126.    Elapsed: 0:12:25.
  Batch 1,900  of  13,126.    Elapsed: 0:13:07.
  Batch 2,000  of  13,126.    Elapsed: 0:13:4

In [42]:
torch.save(model.state_dict(), 'deberta_weights.pth')

In [43]:
model.load_state_dict(torch.load('deberta_weights.pth'))

<All keys matched successfully>

In [44]:
test_model(model)

,-де,-д^е,target_idx,prediction,is_right
331232,додумывавшими,дод^умывавшими,3,дод^умывавшими,1
647800,маринова,мар^инова,3,мар^инова,1
431832,зарядившимся,заряд^ившимся,5,заряд^ившимся,1
424960,заполучаем,заполуч^аем,7,заполуч^аем,1
1643854,шлихи,шлих^и,4,шл^ихи,0
442768,затечешь,затеч^ешь,5,затеч^ешь,1
563471,комплементе,комплем^енте,7,комплем^енте,1
819140,обмельчавшей,обмельч^авшей,7,обмельч^авшей,1
1653022,щурившим,щ^урившим,1,щур^ившим,0
1363772,сдружаетесь,сдруж^аетесь,5,сдруж^аетесь,1


### Заключение

Последние 2 модели слишком долго обучались, поэтому не получилось прогнать их через валидацию (время как на трейне) =((( Но на инференсе видно, что они неплохо обучились, как и bert, accuracy которого составила 0.79